In [4]:
import nltk
import spacy
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import tkinter as tk
from tkinter import scrolledtext


nltk.download('stopwords')


faq_data = [
    {"question": "What is your return policy?", "answer": "Our return policy allows returns within 30 days of purchase."},
    {"question": "How do I track my order?", "answer": "You can track your order using the tracking number provided in the confirmation email."},
    {"question": "Do you offer international shipping?", "answer": "Yes, we offer international shipping to many countries."},
    {"question": "How can I contact customer service?", "answer": "You can contact customer service via email at support@example.com or call us at 123-456-7890."},
    {"question": "What payment methods do you accept?", "answer": "We accept Visa, MasterCard, American Express, and PayPal."},
    {"question": "Can I change or cancel my order?", "answer": "Yes, you can change or cancel your order within 24 hours of placing it."},
    {"question": "How do I reset my password?", "answer": "You can reset your password by clicking on 'Forgot Password' on the login page."},
    {"question": "What is your privacy policy?", "answer": "Our privacy policy ensures that your data is protected and not shared with third parties without your consent."},
    {"question": "Do you have a loyalty program?", "answer": "Yes, we have a loyalty program where you can earn points on every purchase."},
    {"question": "How do I apply a discount code?", "answer": "You can apply a discount code at checkout by entering it in the designated field."}
]


nlp = spacy.load('en_core_web_sm')
stop_words = set(stopwords.words('english'))


def preprocess(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if token.is_alpha and token.text not in stop_words]
    return ' '.join(tokens)


for item in faq_data:
    item["processed_question"] = preprocess(item["question"])

   
vectorizer = TfidfVectorizer()
questions = [item["processed_question"] for item in faq_data]
tfidf_matrix = vectorizer.fit_transform(questions)


def get_most_similar_question(user_question):
    user_question_processed = preprocess(user_question)
    user_question_vector = vectorizer.transform([user_question_processed])
    cosine_similarities = cosine_similarity(user_question_vector, tfidf_matrix)
    most_similar_index = np.argmax(cosine_similarities)
    return faq_data[most_similar_index]["answer"]

def chatbot_response(user_input):
    response = get_most_similar_question(user_input)
    return response


def send():
    user_input = entry.get()
    if user_input.strip():
        chat_log.config(state=tk.NORMAL)
        chat_log.insert(tk.END, "You: " + user_input + "\n")
        entry.delete(0, tk.END)
        response = chatbot_response(user_input)
        chat_log.insert(tk.END, "Bot: " + response + "\n\n")
        chat_log.config(state=tk.DISABLED)
        chat_log.yview(tk.END)


root = tk.Tk()
root.title("Interactive FAQ Chatbot")
root.geometry("500x600")
root.resizable(False, False)


chat_frame = tk.Frame(root, bd=2, bg="pink")
chat_frame.place(relwidth=0.98, relheight=0.8, rely=0.02, relx=0.01)


chat_log = scrolledtext.ScrolledText(chat_frame, state=tk.DISABLED, wrap=tk.WORD, font=("Arial", 12))
chat_log.pack(fill=tk.BOTH, expand=True)


input_frame = tk.Frame(root, bd=2, bg="pink")
input_frame.place(relwidth=0.98, relheight=0.1, rely=0.83, relx=0.01)


entry = tk.Entry(input_frame, font=("Arial", 14))
entry.place(relwidth=0.74, relheight=1)


send_button = tk.Button(input_frame, text="Send", font=("Arial", 14), bg="pink", command=send)
send_button.place(relwidth=0.25, relheight=1, relx=0.75)


root.mainloop()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
